# Lab Assignment 2 
## With this assignment you will get to know more about gradient descent optimization and writing your own functions with forward and backward (i.e., gradient) passes
## You need to complete all the tasks in this notebook in the lab. Edit only those portions in the cells where it asks you to do so!
## Submit **only the notebook file** for evaluation.

### Please Fill these information.
##**Your Name**: Nasif Hossain
##**Your CCID**: 1545143

In [67]:
import torch
from torch.autograd import Variable
from torch.autograd import Function
import torch.nn.functional as F
import numpy as np

## Huber loss function
https://en.wikipedia.org/wiki/Huber_loss

In [68]:
# A loss function measures distance between a predicted and a target tensor
# An implementation of Huber loss function is given below
# We will make use of this loss function in gradient descent optimization
def Huber_Loss(input,delta):
  m = (torch.abs(input)<=delta).detach().float()
  output = torch.sum(0.5*m*input**2 + delta*(1.0-m)*(torch.abs(input)-0.5*delta))
  return output

# Test Huber loss with a couple of different examples

In [69]:
a = torch.tensor([[0.3, 2.0, -3.1],[0.5, 9.2, 0.1]])
print(a.numpy())
ha = Huber_Loss(a,1.0)
print(ha.numpy())

b = torch.tensor([0.3, 2.0])
print(b.numpy())
hb = Huber_Loss(b,1.0)
print(hb.numpy())

[[ 0.3  2.  -3.1]
 [ 0.5  9.2  0.1]]
12.975
[0.3 2. ]
1.545


# Gradient descent code
## Study the following generic gradient descent optimization code.
## Huber loss f measures the distance between a probability vector `z` and target 1-hot vector `target`.
## When `f.backward` is called, PyTorch first computes $\nabla_z f$ (gradient of `f` with respect to `z`), then by chain rule it computes $\nabla_{var} f = J^{z}_{var} \nabla_z f$, where $J^{z}_{var}$ is the Jacobian of `z` with respect to `var`.
## Next, `optimizer.step()` call adjusts the variable `var` in the opposite direction of $\nabla_{var} f.$

In [70]:
def gradient_descent(var,optimizer,softmax,loss,target,nIter,nPrint):
  for i in range(nIter):
    z = softmax(var)
    f = loss(z-target,1.0)
    optimizer.zero_grad()
    f.backward()
    optimizer.step()
    if i%nPrint==0:
      with np.printoptions(precision=3, suppress=True):
        print("Iteration:",i,"Variable:", z.detach().numpy(),"Loss: %0.6f" % f.item())


# Gradient descent with Huber Loss
## The following cell shows how `gradient_descent` function can be used.
## The cell first creates a target 1-hot vector `y`, where only the 3rd place is on.
## It also creates a variable `x` with random initialization and an optimizer.
## Learning rate and momentum has been set to 0.1 and 0.9, respectively.
## Then it calls `gradient_descent` function.

In [71]:
y = torch.zeros(10)
y[2] = 1.0
print("Target 1-hot vector:",y.numpy())
x = torch.randn(y.shape,requires_grad=True)

optimizer = torch.optim.SGD([x], lr=1e-1, momentum=0.9) # create an optimizer that will do gradient descent optimization

gradient_descent(x,optimizer,F.softmax,Huber_Loss,y,1000,100)


Target 1-hot vector: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
Iteration: 0 Variable: [0.113 0.083 0.008 0.013 0.182 0.185 0.051 0.1   0.114 0.152] Loss: 0.559487
Iteration: 100 Variable: [0.121 0.111 0.056 0.018 0.125 0.125 0.081 0.118 0.121 0.124] Loss: 0.500452
Iteration: 200 Variable: [0.007 0.007 0.945 0.002 0.007 0.007 0.006 0.007 0.007 0.007] Loss: 0.001688
Iteration: 300 Variable: [0.005 0.005 0.958 0.001 0.005 0.005 0.004 0.005 0.005 0.005] Loss: 0.000975


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration: 400 Variable: [0.004 0.004 0.965 0.001 0.004 0.004 0.004 0.004 0.004 0.004] Loss: 0.000688
Iteration: 500 Variable: [0.004 0.004 0.969 0.001 0.004 0.004 0.003 0.004 0.004 0.004] Loss: 0.000531
Iteration: 600 Variable: [0.003 0.003 0.972 0.001 0.003 0.003 0.003 0.003 0.003 0.003] Loss: 0.000432
Iteration: 700 Variable: [0.003 0.003 0.974 0.001 0.003 0.003 0.003 0.003 0.003 0.003] Loss: 0.000364
Iteration: 800 Variable: [0.003 0.003 0.976 0.001 0.003 0.003 0.003 0.003 0.003 0.003] Loss: 0.000315
Iteration: 900 Variable: [0.003 0.003 0.978 0.001 0.003 0.003 0.002 0.003 0.003 0.003] Loss: 0.000277


# Let's do a practice on writing differentiable layer using Pytorch's function template

## The cell below implements the following function (squared L2 norm)

$f(x) = ||x||^2$

## Note that x is a vector or a tensor, and f(x) is simply the sum of the square of the elements of x

## The Jacobian of f(x) with respect to x is:

$J_{x}^{f} = 2x$

## The Jacobian here is actually the gradient of $f(x)$ because $f(x)$ outputs a scalar (i.e., single) value

## The backward pass implements: <font color='red'>input_grad = Jacobian x output_grad</font> 


In [72]:
# Inherit from torch.autograd.Function
class My_f(Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    def forward(ctx, x):
        f = torch.sum(x**2)
        ctx.save_for_backward(x,torch.tensor(2.0)) # note that the constant 2.0 is cast as a pytorch tensor before saving
        return f

    @staticmethod
    def backward(ctx, output_grad):
        # retrieve saved tensors and use them in derivative calculation
        x,two = ctx.saved_tensors
        # Return Jacobian-vector product (chain rule)
        input_grad = two*x*output_grad
                
        return input_grad

x = Variable(torch.tensor([3.0,-1.0,0.0,1.0]),requires_grad=True)
my_fval = My_f.apply(x)
print("When x is", x.data, "function value is",my_fval.item())

# compute gradient of f at x
g = torch.autograd.grad(my_fval,x)[0]

print("Gradient of f at x:",g.data)

When x is tensor([ 3., -1.,  0.,  1.]) function value is 11.0
Gradient of f at x: tensor([ 6., -2.,  0.,  2.])


# <font color='red'>20% Weight:</font> In this markdown cell, using [math mode](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html), write gradient of Huber loss function: $output = \sum_i 0.5 m_i (input)^{2}_{i} + \delta (1-m_i)(|input_i|-0.5 \delta)$ with respect to $input.$ Treat $m_i$ to be independent of $input_i,$ because we replaced *if* control statement with $m_i.$
## Your solution : $\frac{\partial (output)}{\partial (input)_i} =  m_i (input_i) + \delta(1-m_i)*sign(input_i)$

# <font color='red'>30% Weight:</font> Define your own (correct!) rule of differentiation for Huber loss function
## Edit indicated line in the cell below. Use the following formula. Do not use for/while/any loop in your solution.
## For this function,  chain rule (Jacobian-vector product) takes the following form: $\frac{\partial (loss)}{\partial (input)_i} = \frac{\partial (output)}{\partial (input)_i} \frac{\partial (loss)}{\partial (output)}.$
# In the `backward` method below, $\frac{\partial (loss)}{\partial (output)}$ is denoted by `output_grad` and the $i^{th}$ component of `input_grad` is symbolized by $\frac{\partial (loss)}{\partial (input)_i}.$

In [73]:
# Inherit from torch.autograd.Function
class My_Huber_Loss(Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    def forward(ctx, input, delta):
        m = (torch.abs(input)<=delta).float()
        ctx.save_for_backward(input,torch.tensor(m),torch.tensor(delta))
        output = torch.sum(0.5*m*input**2 + delta*(1.0-m)*(torch.abs(input)-0.5*delta))
        return output

    @staticmethod
    def backward(ctx, output_grad):
        # retrieve saved tensors and use them in derivative calculation
        input, m, delta = ctx.saved_tensors

        # Return Jacobian-vector product (chain rule)
        # For Huber loss function the Jacobian happens to be a diagonal matrix
        # Also, note that output_grad is a scalar, because forward function returns a scalar value
        input_grad = (m * input + (delta * (1-m))* torch.sign(input)) * output_grad # complete this line, do not use for loop
        # must return two gradients becuase forward function takes in two arguments
        return input_grad, None

# Gradient Descent on Your Own Huber Loss
## You should get almost identical results as before if your rule of differentation is correct!

In [74]:
y = torch.zeros(10)
y[2] = 1.0
print("Target:",y.numpy())
x = Variable(torch.randn(y.shape),requires_grad=True)

optimizer = torch.optim.SGD([x], lr=1e-1, momentum=0.9) # create an optimizer that will do gradient descent optimization

gradient_descent(x,optimizer,F.softmax,My_Huber_Loss.apply,y,1000,100)


Target: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
Iteration: 0 Variable: [0.035 0.115 0.023 0.023 0.032 0.072 0.1   0.057 0.335 0.205] Loss: 0.571437
Iteration: 100 Variable: [0.005 0.009 0.936 0.004 0.005 0.008 0.009 0.007 0.009 0.009] Loss: 0.002280
Iteration: 200 Variable: [0.004 0.006 0.956 0.003 0.003 0.006 0.006 0.005 0.006 0.006] Loss: 0.001091


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Iteration: 300 Variable: [0.003 0.005 0.964 0.002 0.003 0.005 0.005 0.004 0.005 0.005] Loss: 0.000743
Iteration: 400 Variable: [0.003 0.004 0.968 0.002 0.003 0.004 0.004 0.004 0.004 0.004] Loss: 0.000564
Iteration: 500 Variable: [0.002 0.004 0.972 0.002 0.002 0.004 0.004 0.003 0.004 0.004] Loss: 0.000454
Iteration: 600 Variable: [0.002 0.004 0.974 0.002 0.002 0.003 0.003 0.003 0.003 0.003] Loss: 0.000380
Iteration: 700 Variable: [0.002 0.003 0.976 0.001 0.002 0.003 0.003 0.003 0.003 0.003] Loss: 0.000326
Iteration: 800 Variable: [0.002 0.003 0.977 0.001 0.002 0.003 0.003 0.003 0.003 0.003] Loss: 0.000286
Iteration: 900 Variable: [0.002 0.003 0.979 0.001 0.002 0.003 0.003 0.002 0.003 0.003] Loss: 0.000254


# <font color='red'>20% Weight:</font> In this markdown using math mode write Jacobian of softmax function: $(output)_i = \frac{exp((input)_i)}{ \sum_j exp((input)_j)}.$
## Your solution : 
\begin{equation*}
    \frac{\partial (output)_j}{\partial (input)_i} = \begin{cases}
               {output_i*(1-output_j)},               & i = j,\\
               -{output_i*output_j} , & \text{otherwise.}
           \end{cases}
\end{equation*}
# Putting them together:
\begin{equation*}
\frac{\partial (output)_j}{\partial (input)_i} = output_i(1(i=j) - output_j(i!= j)) 
\end{equation*}
\begin{equation*} 
  or, \frac{\partial (output)_j}{\partial (input)_i} = output_i(i=j) - output_i * output_j (i != j)
\end{equation*}
\begin{equation*} 
  Therefore, \frac{\partial (output)_j}{\partial (input)_i}= diagonal(output) - outerproduct(outer, outer) 
\end{equation*}


# <font color='red'>30% Weight:</font> Your own softmax with forward and backward functions
## Edit indicated line in the cell below. Use the following formula. Do not use for/while/any loop in your solution.
## The Jacobian-vector product (chain rule) takes the following form using summation sign: $\frac{\partial (loss)}{\partial (input)_i} = \sum_j \frac{\partial (output)_j}{\partial (input)_i} \frac{\partial (loss)}{\partial (output)_j}$
# Once again note that, in the `backward` method below, $i^{th}$ component of `input_grad` and $j^{th}$ component of `output_grad` are denoted by $\frac{\partial (loss)}{\partial (input)_i}$ and $\frac{\partial (loss)}{\partial (output)_j}$, respectively.

In [75]:
# Inherit from Function
class My_softmax(Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    def forward(ctx, input):
        output = F.softmax(input,dim=0)
        ctx.save_for_backward(output) # this is the only tensor you will need to save for backward function
        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, output_grad):
        output = ctx.saved_tensors[0]
        # retrieve saved tensors and use them in derivative calculation
        # return Jacobian-vecor product
        input_grad = torch.sum((torch.diag(output) - torch.outer(output, output)) * output_grad,1)  # Complete this line
        return input_grad

# Gradient Descent on your own Huber Loss and your own softmax

In [76]:
y = torch.zeros(10)
y[2] = 1.0
print(y)
x = Variable(torch.randn(y.shape),requires_grad=True)
print(x)

optimizer = torch.optim.SGD([x], lr=1e-1, momentum=0.9) # create an optimizer that will do gradient descent optimization

gradient_descent(x,optimizer,My_softmax.apply,My_Huber_Loss.apply,y,1000,100)


tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor([-1.2793, -1.2194, -0.4673, -2.5135, -0.3498,  0.9110,  0.4048,  0.5329,
         0.7712,  0.2621], requires_grad=True)
Iteration: 0 Variable: [0.025 0.027 0.056 0.007 0.063 0.223 0.135 0.153 0.194 0.117] Loss: 0.519346
Iteration: 100 Variable: [0.003 0.003 0.947 0.001 0.006 0.008 0.008 0.008 0.008 0.008] Loss: 0.001618
Iteration: 200 Variable: [0.002 0.002 0.959 0.001 0.005 0.006 0.006 0.006 0.006 0.006] Loss: 0.000943
Iteration: 300 Variable: [0.002 0.002 0.966 0.001 0.004 0.005 0.005 0.005 0.005 0.005] Loss: 0.000668


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Iteration: 400 Variable: [0.002 0.002 0.97  0.001 0.003 0.005 0.005 0.005 0.005 0.004] Loss: 0.000517
Iteration: 500 Variable: [0.002 0.002 0.973 0.    0.003 0.004 0.004 0.004 0.004 0.004] Loss: 0.000422
Iteration: 600 Variable: [0.001 0.002 0.975 0.    0.003 0.004 0.004 0.004 0.004 0.004] Loss: 0.000356
Iteration: 700 Variable: [0.001 0.001 0.977 0.    0.003 0.004 0.003 0.004 0.004 0.003] Loss: 0.000308
Iteration: 800 Variable: [0.001 0.001 0.978 0.    0.003 0.003 0.003 0.003 0.003 0.003] Loss: 0.000271
Iteration: 900 Variable: [0.001 0.001 0.979 0.    0.002 0.003 0.003 0.003 0.003 0.003] Loss: 0.000242
